In [11]:
from gbmi.exp_modular_fine_tuning.train import MODULAR_ADDITION_113_CLOCK_CONFIG
from gbmi.exp_modular_fine_tuning.train import ModularFineTuningTrainingWrapper
from gbmi.model import train_or_load_model
import torch
import einops
from torch import tensor
from math import *


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
config = MODULAR_ADDITION_113_CLOCK_CONFIG
runtime, model = train_or_load_model(config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb:   1 of 1 files downloaded.  
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [5]:
loss_fn = ModularFineTuningTrainingWrapper.loss_fn
for param in model.parameters():
    param.requires_grad = False
embedding_matrix = model.embed.W_E.detach().clone()
embedding_matrix.requires_grad = False
model.embed = torch.nn.Identity()

In [40]:
p = 113
q = 2*p
class DifferentModClock(torch.nn.Module):
    def __init__(self,q):
        super(DifferentModClock,self).__init__()
        self.W_e = torch.nn.Parameter(-0.5/sqrt(p+1) + torch.rand((q+1,p+1))/sqrt(p+1)).to('cuda')
        self.W_u = torch.nn.Parameter(-0.5/sqrt(p) + torch.rand((q,p))/sqrt(p)).to('cuda')
    def forward(self,x):
        z = torch.nn.functional.one_hot(x.to('cuda')).to(torch.float)
        
        y = z @ self.W_e @ embedding_matrix
        if(len(y.shape)!=3):
            y = y.unsqueeze(0)
        return self.W_u @ (einops.rearrange(model(y),"b d p -> b p d"))
Clock = DifferentModClock(q)
print(Clock(torch.tensor([[1,2,226],[2,5,226]])).shape)

torch.Size([2, 226, 3])


In [53]:
a_vector = einops.repeat(torch.arange(q), "i -> (i j)", j=q)
b_vector = einops.repeat(torch.arange(q), "j -> (i j)", i=q)
equals_vector = einops.repeat(torch.tensor(q), " -> (i j)", i=q, j=q)
dataset = torch.stack([a_vector, b_vector, equals_vector], dim=1).cuda()


In [62]:
def run_batch(x):
        labels = (x[:, 0] + x[:, 1]) % q
        y_preds = Clock(x)

        loss = loss_fn(y_preds, labels)
        self.log(f"{prefix}loss", loss, prog_bar=True)
        return loss

In [64]:
run_batch(dataset)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
